<a href="https://colab.research.google.com/github/daucel/Proyecto1/blob/main/Tesla_GME_Q1_to_Q6_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Tesla and GameStop Stock Analysis — Q1 to Q6

.


## Setup

In [ ]:

# If needed, uncomment to install:
# !pip install yfinance requests beautifulsoup4 lxml plotly pandas

import re, json
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from io import StringIO
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 140)
print("✅ Libraries ready")


## Helper Functions

In [ ]:

def scrape_macrotrends_quarterly_revenue_fast(ticker, slug):
    """Scrape quarterly revenue from Macrotrends by extracting the embedded JSON in a script tag."""
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{slug}/revenue"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=30).text

    # Primary: exact variable 'quarterly_revenue'
    m = re.search(r"var\s+quarterly_revenue\s*=\s*(\[.*?\]);", html, re.S)
    if not m:
        # Fallback: any JSON-like array containing 'revenue' keys
        m = re.search(r"(\[\s*{[^\]]+\brevenue\b[^\]]+\])", html, re.S)
    if not m:
        raise RuntimeError("Quarterly revenue JSON not found on page.")

    data = json.loads(m.group(1))
    df = pd.DataFrame(data)

    # Normalize columns
    date_col = "date" if "date" in df.columns else ("Date" if "Date" in df.columns else None)
    rev_col  = "revenue" if "revenue" in df.columns else ("Revenue" if "Revenue" in df.columns else None)
    if not date_col or not rev_col:
        raise RuntimeError("Unexpected JSON structure for revenue.")

    df = df[[date_col, rev_col]].rename(columns={date_col: "Date", rev_col: "Revenue"})
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Revenue"] = pd.to_numeric(df["Revenue"].astype(str).str.replace(r"[$,]", "", regex=True), errors="coerce")
    df = df.dropna().sort_values("Date").reset_index(drop=True)
    return df

def yf_quarterly_revenue(ticker):
    """Fallback: derive quarterly revenue from yfinance fundamentals (income statement)."""
    q = yf.Ticker(ticker).quarterly_income_stmt.T
    for col in ["Total Revenue", "TotalRevenue", "Revenue"]:
        if col in q.columns:
            df = (q[[col]]
                  .reset_index()
                  .rename(columns={"index": "Date", col: "Revenue"}))
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
            df = df.dropna().sort_values("Date").reset_index(drop=True)
            return df
    raise RuntimeError("Revenue column not found in yfinance fundamentals.")

def make_graph(stock_data, title):
    fig = go.Figure(data=go.Scatter(x=stock_data["Date"], y=stock_data["Close"], mode="lines", name="Close"))
    fig.update_layout(title=title, xaxis_title="Date", yaxis_title="Close Price (USD)", template="plotly_white", height=500)
    fig.show()


## Q1 — Use yfinance to Extract Tesla Stock Data

In [ ]:

# Download Tesla stock data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
# Reset the index
tesla_data.reset_index(inplace=True)
# Display first 5 rows for screenshot
tesla_data.head()


## Q2 — Use Webscraping to Extract Tesla Revenue Data (show last 5 rows)

In [ ]:

# Attempt robust web scraping; fallback to yfinance fundamentals if scraping fails
try:
    tesla_revenue = scrape_macrotrends_quarterly_revenue_fast("TSLA", "tesla")
except Exception as e:
    print("Scraper failed, using yfinance fundamentals fallback. Reason:", e)
    tesla_revenue = yf_quarterly_revenue("TSLA")

# Show last five rows for screenshot
tesla_revenue.tail()


## Q3 — Use yfinance to Extract GameStop Stock Data

In [ ]:

# Download GameStop stock data
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
# Reset the index
gme_data.reset_index(inplace=True)
# Display first 5 rows for screenshot
gme_data.head()


## Q4 — Use Webscraping to Extract GME Revenue Data (show last 5 rows)

In [ ]:

# Attempt robust web scraping; fallback to yfinance fundamentals if scraping fails
try:
    gme_revenue = scrape_macrotrends_quarterly_revenue_fast("GME", "gamestop")
except Exception as e:
    print("Scraper failed, using yfinance fundamentals fallback. Reason:", e)
    gme_revenue = yf_quarterly_revenue("GME")

# Show last five rows for screenshot
gme_revenue.tail()


## Q5 — Plot Tesla Stock Graph

In [ ]:

make_graph(tesla_data, "Tesla (TSLA) — Close Price Over Time")


## Q6 — Plot GameStop Stock Graph

In [ ]:

make_graph(gme_data, "GameStop (GME) — Close Price Over Time")
